## How to generate a synthetic dataset 
For example, use 5% of real data to train pix2pix network and then generate synthetic data.

In [ ]:
# Augmentation networks
# train pix2pix with real data
%run ./scripts/train_pix2pix.py   --name iron_label2real_pix2pix_real_5 \
--gpu_ids 0,1 --batch_size 8 --checkpoints_dir './ckpts/aug_models' \
--dataroot './datasets/' --phase 'train' --label_dir 'real_labels' --image_dir 'real_images' \
--record_txt './datasets/train/records_txt/real_5.txt' \
--display_port 6006 --display_env 'real_5'

In [ ]:
# generate synthetic images with synthctic labels
%run ./scripts/generate_samples.py  --name iron_label2real_pix2pix_real_5  --num_test 28800 \
--gpu_ids 0 --batch_size 1 --checkpoints_dir './ckpts/aug_models/' \
--dataroot './datasets/' --phase 'train' --label_dir 'syn_labels' \
--results_dir './datasets/train/syn_images'  --epoch 'latest'

In [ ]:
# check dataset
%matplotlib inline
%run ./scripts/check_paired_dataset.py --check_num 10 \
--images_path './datasets/train/iron_label2real_pix2pix_real_35/latest/images' \
--labels_path './datasets/train/iron_label2real_pix2pix_real_35/latest/labels/' 

## Applying in Image segmentations
Use real and synthetic data above as data augmentation to train U-Net for Image segmentation

In [ ]:
# our mix train method for image segmentation
# train unet with synthetic data
%run ./scripts/train.py --net unet --train_type 'mix' --gpu_ids 0,1 \
--dataroot './datasets/' --batch_size 8 \
--train_img_dir_real 'train/real_images' \
--train_label_dir_real 'train/real_labels' \
--train_img_list_real './datasets/train/records_txt/real_5.txt' \
--train_img_dir_syn 'train/syn_images/iron_label2real_pix2pix_real_5/latest' \
--train_label_dir_syn 'train/syn_labels' \
--val_img_dir 'val/real_images' \
--val_label_dir 'val/real_labels' \
--checkpoints_dir './ckpts/seg_models' \
--name 'model_real_5_mix_syn' \
--display_env 'model_real_5_mix_syn' \
--display_port 6006 

In [ ]:
# test our method
%run ./scripts/test.py --net unet --name 'model_real_5_mix_syn'\
--checkpoints_dir './ckpts/seg_models/' \
--dataroot './datasets/test/' \
--test_img_dir 'real_images' \
--test_label_dir 'real_labels' \
--epoch 'epoch_4_7200' --results_dir './results/' --gpu_ids 0

## Compare to other methods

In [ ]:
# unet
# origin train 
%run ./scripts/train.py --net unet --train_type 'origin' --batch_size 8 --gpu_ids 0,1 \
--dataroot './datasets/' \
--train_img_dir_real 'train/real_images' \
--train_label_dir_real 'train/real_labels' \
--train_img_list_real './datasets/train/records_txt/real_5.txt' \
--val_img_dir 'val/real_images' \
--val_label_dir 'val/real_labels' \
--checkpoints_dir './ckpts/seg_models' \
--name 'model_real_5' \
--display_env 'model_real_5' \
--display_port 6006 

In [ ]:
# test unet
%run ./scripts/test.py --net unet --name 'model_real_5'\
--checkpoints_dir './ckpts/seg_models/' \
--dataroot './datasets/test/' \
--test_img_dir 'real_images' \
--test_label_dir 'real_labels' \
--results_dir './results/' --gpu_ids 0

In [ ]:
# ternausnet with pretrained vgg11
# train
%run ./scripts/train.py --net unet11 --train_type 'origin' --batch_size 8 --gpu_ids 0,1 \
--dataroot './datasets/' \
--train_img_dir_real 'train/real_images' \
--train_label_dir_real 'train/real_labels' \
--train_img_list_real './datasets/train/records_txt/real_5.txt' \
--val_img_dir 'val/real_images' \
--val_label_dir 'val/real_labels' \
--checkpoints_dir './ckpts/seg_models' \
--name 'model_real_5_pretrained_vgg11' \
--display_env 'model_real_5_pretrained_vgg11' \
--display_port 6006 

In [ ]:
# test ternausnet
%run ./scripts/test.py --net unet11 --name 'model_real_5_pretrained_vgg11'\
--checkpoints_dir './ckpts/seg_models/' \
--dataroot './datasets/test/' \
--test_img_dir 'real_images' \
--test_label_dir 'real_labels' \
--epoch 'best_val' --results_dir './results/' --gpu_ids 0

In [ ]:
# auto image segmentation by threshold, morphology and edge detection
# otsu|canny|watershed|kmeans...
%run ./scripts/auto_segmentations.py --images_path './datasets/test/real_images/' --labels_path './datasets/test/real_labels/' \
--results_dir './results/others' --method 'kmeans'